## 패키지 설치

In [ ]:
!pip install torch
!pip install transformers
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf' : command 에서 실행!

#!pip install sentencepiece
#pip install transformers==4.8.2
#pip install mxnet
#pip install importlib_metadata==7.1.0
#pip install pandas
#pip install scikit-learn

## 패키지 임포트

In [6]:
from transformers import BertModel
from kobert_tokenizer import KoBERTTokenizer
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
import re
from sklearn.metrics import f1_score, accuracy_score
from torch.nn import functional as F
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model = BertModel.from_pretrained('skt/kobert-base-v1')

## 데이터 로드 및 분할

In [7]:
data = pd.read_csv('./train_demo.csv')
#train & test 데이터로 나누기

data_train, data_test = train_test_split(data, test_size=0.2, random_state=0)
data_train.reset_index(drop=True, inplace=True)
data_test.reset_index(drop=True, inplace=True)

## 커스텀 데이터셋 정의

In [8]:
# 커스텀 데이터셋 정의
class KoBERTDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # 데이터 프레임의 텍스트 열을 결합하여 하나의 텍스트로 만듭니다.
        text = self.dataframe.loc[idx, ['origin_title', 'origin_content', 'relative_title', 'relative_content']].str.cat(sep='. ')
        sentences = re.split(r'\.\s+|\.\\n+|\.$', text)
        label = self.dataframe.loc[idx, 'label']

        # 빈 문자열 제거
        sentences = [sentence.strip() for sentence in sentences if sentence]

        all_input_ids = ''
        # 각 문장을 인코딩하고 디코딩하여 확인
        for st in sentences:
            encoding = tokenizer.encode_plus(
                st,
                add_special_tokens=True,
            )
            all_input_ids+=(tokenizer.decode(encoding['input_ids']))

        # 모든 문장을 하나의 입력으로 합치기 위한 리스트 초기화

        # 각 문장을 인코딩하고 결과를 리스트에 추가
        encoding = tokenizer.encode_plus(
                    all_input_ids,
                    add_special_tokens=False,
                    max_length=self.max_len,
                    return_token_type_ids=False,
                    padding='max_length',
                    truncation=True,
                    return_attention_mask=True,
                    return_tensors='pt',
                )
        # encoding = self.tokenizer.encode_plus(
        #     text,
        #     add_special_tokens=True,
        #     max_length=self.max_len,
        #     return_token_type_ids=False,
        #     padding='max_length',
        #     truncation=True,
        #     return_attention_mask=True,
        #     return_tensors='pt',
        # )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

## 학습

In [9]:
# 이진 분류를 위한 모델 정의
class KoBERTClassifier(nn.Module):
    def __init__(self, bert_model, num_classes):
        super(KoBERTClassifier, self).__init__()
        self.bert = bert_model
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=False
        )
        output = self.drop(pooled_output)
        return self.out(output)

# KoBERT 모델과 토크나이저 로드
kobert_tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model = BertModel.from_pretrained('skt/kobert-base-v1')

# 데이터셋과 데이터 로더 준비
max_len = 512
batch_size = 2
dataset = KoBERTDataset(dataframe=data_train, tokenizer=kobert_tokenizer, max_len=max_len) # data 입력
data_loader = DataLoader(dataset, batch_size=batch_size)

# 분류 모델 초기화
num_classes = 2  # 이진 분류
bert_model = BertModel.from_pretrained('skt/kobert-base-v1')
classifier_model = KoBERTClassifier(bert_model, num_classes)

# 모델 학습을 위한 준비
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classifier_model = classifier_model.to(device)
optimizer = optim.Adam(classifier_model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

# 모델 학습 (간단한 예시)
classifier_model.train()
for epoch in range(2):  # 실제 사용 시 에폭 수 조정 필요
    total_loss =0
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = classifier_model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print(f"Epoch {epoch}, Loss: {loss.item()}")
        total_loss +=loss.item()
    print(f"Total Epoch {epoch+1}, Loss: {total_loss}")


Total Epoch 1, Loss: 2.2725155353546143
Total Epoch 2, Loss: 1.967287838459015


## 예측

In [10]:
data_test = data_test # 실제 사용 시에는 테스트 데이터셋으로 변경해야 합니다.
test_dataset = KoBERTDataset(dataframe=data_test, tokenizer=tokenizer, max_len=max_len)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# 모델을 평가 모드로 설정
classifier_model.eval()

# 예측을 저장할 리스트
predictions = []
# 실제 레이블을 저장할 리스트
true_labels = []

# 모델을 평가 모드로 설정
classifier_model.eval()

# 테스트 데이터에 대한 예측 수행 및 실제 레이블 수집
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device) # 이번에는 실제 레이블도 수집합니다.

        outputs = classifier_model(input_ids, attention_mask)
        probs = F.softmax(outputs, dim=1)
        pred = torch.argmax(probs, dim=1)
        predictions.extend(pred.cpu().tolist())
        true_labels.extend(labels.cpu().tolist())

# F1 스코어와 정확도 계산
f1 = f1_score(true_labels, predictions, average='weighted')  # 다중 클래스 문제의 경우 'weighted' 사용
accuracy = accuracy_score(true_labels, predictions)

# 성능 출력
print(f'F1 Score: {f1}')
print(f'Accuracy: {accuracy}')

F1 Score: 0.3333333333333333
Accuracy: 0.5
